# DATA620: Final Project Proposal - Federal Reserve Sentiment

## Homework Team 3: David Simbandumwe, Eric Lehmphul and Lidiia Tronina

Your proposal should describe at a high level what you’re seeking to accomplish, and your motivation for performing this analysis.  A guiding question or hypothesis to test is one good way to start.  If you are going to work in a small group (encouraged!), you should also list your partners’ names. 
You should briefly describe your data sources, plan for doing the work, and up front concerns.  If you are working in a group, please describe the roles and responsibilities of each group member.
We’ll treat this proposal as a planning document, not a blue print containing “firm, fixed requirements.”
Your final project proposal is due on Monday 12/9. 

## Introduction

How consistent is the messaging leading up to a decision by Federal Reserve Open Market Committee (FOMC)


#### Federal Reserve System

Federal Reserve Act, created The Federal Reserve System on December 23, 1913. The Federal Reserve acts as the central banking system of the United States of America. The Federal Reserve's system has 3 primary components 
- 12 Federal Reserve Banks 
- Federal Reserve Board of Governors
- Federal Open Market Committee

This a unique structure among central banks in that it includes both public and private characteristics. 

The Fed has many key functions however its activities can be categorized by the Dual Mandate setup by the Congress in the Federal Reserve Act. 
- maximum employment
- stable prices
he dual mandate is the long-run goal for monetary policy, and the Congress also established the Federal Reserve as an independent agency to help ensure that this monetary policy goal can be achieved.

To support these objectives the federal reserve employs monetary policy through 3 primary mechanisms
- open market operations
- the discount rate
- reserve requirements


The Fed has evolved from days of Alen Greenspan when being opaque was a primary focus. How well does the Fed achieve these goals

For this analysis we will focus on the Feds Open Market Operations as carried out by the Federal Open Market Committee and the official communications by FOMC members leading up to official FOMC meetings. At each FOMC meeting the members of the committee will reach consensus on the appropriate monitory policy course. This consensus takes the form a formal communication and directive to the Federal Reserve Bank of New York to execute the transactions on behalf of the Federal Reserve System. 


## Hypothesis
We can categorize the periods between FOMC meeting by their outcome. In each meeting the interest rate target will either be increased, decreased or remain the same. For each type of FOMC meeting outcome we would like to understand if the communication by the representatives of the Fed is consistent. 

- H0 - Communications by Federal Reserve Board Governs do not telegraph the outcome of the upcoming FOMC Meeting. There is no decipherable difference in speeches by FOMC members during periods of increase, decreasing or stable interest rates. 

- H1 - Federal Reserve Board Governs will telegraph the outcome of the upcoming FOMC Meeting through their communications. The speeches by FOMC members differ during periods of increase, decreasing or stable interest rates.  

## Data Source
There are several sources of prepossessed data from the fed however it might be necessary to augment these data set to get the latest

1. Aggregate FOMC Decisions - the outcome of each FOMC meeting is captured in a broader write-up however the decision can be distilled into meeting date, meeting location, decision to increase, decrease or leave interested unchanged, and a magnitude of the change, and the new target range for interest rates
2. Aggregate Speeches by FOMC Members - Each member of the FOMC maintains a detailed calendar of speeches that includes, location, audience, speech title, speech text.

<br>
<b>Potential Sources</b>

Meeting calendars, statements, and minutes (2017-2022)
https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm
https://www.thebalance.com/fomc-meetings-schedule-and-statement-summaries-3305975

Federal Funds Effective Rate (FEDFUNDS)
https://fred.stlouisfed.org/series/FEDFUNDS

List of Governors and their terms
https://en.wikipedia.org/wiki/Federal_Reserve_Board_of_Governors

History of FOMC decisions 
https://www.federalreserve.gov/monetarypolicy/fomc.htm
https://en.wikipedia.org/wiki/History_of_Federal_Open_Market_Committee_actions

## Analysis

Review speeches by federal reserve board members. Analysis of Speech Sentiment for consistency with the decision making. Does the Federal reserve do a good job of telegraphing its decisions in speeches leading up to the meeting of the?

1. The outcomes of the FOMC meeting will be used to label speeches by Board Members, Each speech takes place during a period where the FOMC board will decide to increase, decrease or leave interest rates unchanged. The speech can then be parsed for and scored using sentiment analysis

2. The records of the speech will be used to create a network graph where the nodes will be speaker, audience, location?. The edges will be sentiment and the meta data from the relevant FOMC meeting.

3. Grouping the communications by FOMC Meeting outcome, Speaker, Audience, Time frame we can analyze the clarity of message from the Federal Reserve.



## Concerns

## Final Thoughts

## Project Plan

### Team Roles

### Work Plan

## Other Links:
    
- https://www.youtube.com/watch?v=A9bQzmE6QGw

- https://www.youtube.com/watch?v=VzMiRsSOctI&t=1487s